In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import json
import splinter
import requests

In [2]:
# Scrape Monster.com

In [3]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)


# URL of search page
url = "https://www.monster.com/jobs/search/?q=Data-Scientist&where=California"
# Visit URL with ChromeDriver
browser.visit(url)

companies = []
titles = []
locations = []
urls = []

# Loop to load more jobs
for i in range(9):
    # Create bs object for each page
    soup = bs(browser.html, "html.parser")
    # Scrape for parent tag 
    
    try:
        summary = soup.find_all('div', class_="summary")
    
        # Scrape child tags for corresponding information
        # company name
        for x in summary:
            try:
                companies.append(x.find('div', class_="company").text.replace('\n', ''))
            except:
                companies.append("NaN")
            
            try:
                titles.append(x.find('h2', class_="title").text.replace('\n', ''))
            except:
                titles.append("NaN")
            
            try:
                locations.append(x.find('div', class_="location").text.replace('\n', ''))
            except:
                locations.append("NaN")
                
            try:
                urls.append(x.find('h2', class_="title").a.get('href'))
            except:
                urls.append("NaN")
    except:
        print("empty page")
    # click 'Load more jobs' button with ChromeDriverr
    browser.click_link_by_partial_text('Load more jobs')


which: no chromedriver in (/c/Users/fangq/bin:/mingw64/bin:/usr/local/bin:/usr/bin:/usr/bin:/mingw64/bin:/usr/bin:/c/Users/fangq/bin:/c/Program Files (x86)/Intel/Intel(R) Management Engine Components/iCLS:/c/Program Files/Intel/Intel(R) Management Engine Components/iCLS:/c/windows/system32:/c/windows:/c/windows/System32/Wbem:/c/windows/System32/WindowsPowerShell/v1.0:/c/Program Files (x86)/NVIDIA Corporation/PhysX/Common:/c/Program Files (x86)/Intel/Intel(R) Management Engine Components/DAL:/c/Program Files/Intel/Intel(R) Management Engine Components/DAL:/c/Program Files (x86)/Intel/Intel(R) Management Engine Components/IPT:/c/Program Files/Intel/Intel(R) Management Engine Components/IPT:/cmd:/c/Program Files/Microsoft VS Code/bin:/c/WINDOWS/system32:/c/WINDOWS:/c/WINDOWS/System32/Wbem:/c/WINDOWS/System32/WindowsPowerShell/v1.0:/c/WINDOWS/System32/OpenSSH:/c/Program Files/MySQL/MySQL Utilities 1.6:/c/Users/fangq/Anaconda3:/c/Users/fangq/Anaconda3/Library/mingw-w64/bin:/c/Users/fangq/An

In [4]:
len(titles)

500

In [6]:
monster_df =  pd.DataFrame({
    'Company': companies,
    'Title': titles,
    'Location': locations,
    'URL': urls
})

# Delete possible duplicates and clean up
monster_df.drop_duplicates(subset='URL', inplace=True)
monster_df.drop_duplicates(subset=['Company','Title','Location'], inplace=True)
monster_df.dropna()
monster_df = monster_df.reset_index(drop=True)

# for merging with other data set
# add column 'Salary' and set to blank
monster_df['Salary'] = " "
# add column 'Site' and fill in "monster"
monster_df['Site'] = "Monster"
# add column 'Keyword'
monster_df['Keyword'] = "Data Scientist"
# split location into city and state
monster_df['City'] = monster_df['Location'].str.split(',', expand=True)[0]
monster_df['State'] = monster_df['Location'].str.split(',', expand=True)[1]

len(monster_df)

123

In [7]:
monster_df

,Company,Title,Location,URL,Salary,Site,Keyword,City,State
0,Booz Allen Hamilton,Corporate Data Scientist Program Assessor,"El Segundo, CA",https://job-openings.monster.com/corporate-dat...,,Monster,Data Scientist,El Segundo,CA
1,Eclaro,Data Scientist,"San Jose, CA",https://job-openings.monster.com/data-scientis...,,Monster,Data Scientist,San Jose,CA
2,The Judge Group,Data Scientist III,"Rancho Cordova, CA",https://job-openings.monster.com/data-scientis...,,Monster,Data Scientist,Rancho Cordova,CA
3,OSI Engineering,Data Scientist for a HOT AI Analytics Startup ...,"San Mateo, CA",https://job-openings.monster.com/data-scientis...,,Monster,Data Scientist,San Mateo,CA
4,Pfizer,Clinical Data Scientist (Manager) [Mandarin pr...,"San Diego, CA",https://job-openings.monster.com/clinical-data...,,Monster,Data Scientist,San Diego,CA
5,Bosch,Data Scientist,"Palo Alto, CA",https://job-openings.monster.com/data-scientis...,,Monster,Data Scientist,Palo Alto,CA
6,CyberCoders,Lead Data Scientist -- Thriving Leader in the ...,"San Francisco, CA",https://job-openings.monster.com/lead-data-sci...,,Monster,Data Scientist,San Francisco,CA
7,Personiv,Data Scientist,"Mountain View, CA",https://job-openings.monster.com/data-scientis...,,Monster,Data Scientist,Mountain View,CA
8,Insight Global,Data Scientist,"Burbank, CA",https://job-openings.monster.com/data-scientis...,,Monster,Data Scientist,Burbank,CA
9,Johnson Controls,Data Scientist I,"Santa Clara, CA",https://job-openings.monster.com/data-scientis...,,Monster,Data Scientist,Santa Clara,CA


In [8]:
# Define a list to store description of each job listing
descs = []
# Counter
i = 0
# Loop through all the urls in the list
for url in monster_df['URL'].tolist():
    # Visit each URL
    browser.visit(url)
    # Create bs object for each URL
    soup = bs(browser.html, "html.parser")
    
    # Scrape description only when exists 
    try:
        desc = soup.find('div', id="JobDescription").text
        print(i)
    
    except AttributeError as e:
        desc = " "
    
    # Iteration completed, add 1 to counter
    i += 1
    # Append result to list
    descs.append(desc)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122


In [9]:
# add and clean up descriptions
monster_df['Description']= descs
monster_df['Description'] = monster_df['Description'].str.strip('\n')
monster_df

,Company,Title,Location,URL,Salary,Site,Keyword,City,State,Description
0,Booz Allen Hamilton,Corporate Data Scientist Program Assessor,"El Segundo, CA",https://job-openings.monster.com/corporate-dat...,,Monster,Data Scientist,El Segundo,CA,Job Number: R0031021Corporate Data Scientist P...
1,Eclaro,Data Scientist,"San Jose, CA",https://job-openings.monster.com/data-scientis...,,Monster,Data Scientist,San Jose,CA,Reference # : 18-08901Title : Data ScientistLo...
2,The Judge Group,Data Scientist III,"Rancho Cordova, CA",https://job-openings.monster.com/data-scientis...,,Monster,Data Scientist,Rancho Cordova,CA,THIS IS A PERMANENT POSITION - ONLY UCS & GC A...
3,OSI Engineering,Data Scientist for a HOT AI Analytics Startup ...,"San Mateo, CA",https://job-openings.monster.com/data-scientis...,,Monster,Data Scientist,San Mateo,CA,Summary:You will be working with a team that f...
4,Pfizer,Clinical Data Scientist (Manager) [Mandarin pr...,"San Diego, CA",https://job-openings.monster.com/clinical-data...,,Monster,Data Scientist,San Diego,CA,As part of the Data Monitoring and Management ...
5,Bosch,Data Scientist,"Palo Alto, CA",https://job-openings.monster.com/data-scientis...,,Monster,Data Scientist,Palo Alto,CA,"Data Scientist384 Santa Trinita Ave, Sunnyvale..."
6,CyberCoders,Lead Data Scientist -- Thriving Leader in the ...,"San Francisco, CA",https://job-openings.monster.com/lead-data-sci...,,Monster,Data Scientist,San Francisco,CA,If you are a Lead Data Scientist with experien...
7,Personiv,Data Scientist,"Mountain View, CA",https://job-openings.monster.com/data-scientis...,,Monster,Data Scientist,Mountain View,CA,Position: Data ScientistWork Type: Full TimeRe...
8,Insight Global,Data Scientist,"Burbank, CA",https://job-openings.monster.com/data-scientis...,,Monster,Data Scientist,Burbank,CA,An employer is looking for a Data Scientist in...
9,Johnson Controls,Data Scientist I,"Santa Clara, CA",https://job-openings.monster.com/data-scientis...,,Monster,Data Scientist,Santa Clara,CA,"{""companyInfo"":{""yearF..."


In [11]:
#monster_df.to_csv("Data/monster_data_science.csv")